In [25]:
import pandas as pd
import numpy as np

# import seaborn as sns
# from matplotlib.pyplot import figure

# For Gloria
import gloria
from gloria import Gloria, cross_validation, performance_metrics

# import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
# import cmdstanpy; cmdstanpy.install_cmdstan(overwrite=True, version = '2.36.0')

In [2]:
df = pd.read_csv(r"C:\Users\nvanderweerd\OneDrive - Cordaan\Documenten\Overig\test Gloria\2024-11-21_poisson_test_n00.csv")
df

,Unnamed: 0,ds,y,y_yhat,trend,is_changepoint,is_anomaly,ano_deviation
0,0,2020-01-01,20,20.515059,19.298470,False,False,0
1,1,2020-01-02,19,20.984836,19.308590,False,False,0
2,2,2020-01-03,19,21.092862,19.318715,False,False,0
3,3,2020-01-04,24,20.915238,19.328845,False,False,0
4,4,2020-01-05,22,20.635167,19.338980,False,False,0
...,...,...,...,...,...,...,...,...
360,360,2020-12-26,40,36.659458,18.037091,False,False,0
361,361,2020-12-27,43,38.347244,18.014367,False,False,0
362,362,2020-12-28,24,35.152067,17.991673,False,False,0
363,363,2020-12-29,25,27.694075,17.969006,False,False,0


In [3]:
df['ds'] = pd.to_datetime(df['ds'])
df['y'] = df['y'].astype('uint')
df = df[['ds', 'y']]
df.dtypes

ds    datetime64[ns]
y             uint32
dtype: object

In [15]:
model = Gloria(
    model = 'poisson'
    , sampling_period = '1 day'
    , timestamp_name = 'ds'
    , metric_name = 'y'
    , n_changepoints = 0
)

model.add_seasonality(name='monthly', period=30.5, fourier_order=12)

07:01:26 - cmdstanpy - INFO - compiling stan file C:\Users\nvanderweerd\Documents\gloria\gloria\stan_models\poisson.stan to exe file C:\Users\nvanderweerd\Documents\gloria\gloria\stan_models\poisson.exe
07:12:05 - cmdstanpy - INFO - compiled model executable: C:\Users\nvanderweerd\Documents\gloria\gloria\stan_models\poisson.exe


Gloria(model='poisson', sampling_period=Timedelta('1 days 00:00:00'), timestamp_name='ds', metric_name='y', capacity_name='', changepoints=None, n_changepoints=0, changepoint_range=0.8, seasonality_prior_scale=3, event_prior_scale=3, changepoint_prior_scale=3, dispersion_prior_scale=3, interval_width=0.8, trend_samples=1000, model_backend=<gloria.models.Poisson object at 0x0000020933033B90>, vectorized=False, external_regressors={}, seasonalities={'monthly': Seasonality(name='monthly', prior_scale=3.0, period=3.472222222222222e-13, fourier_order=12)}, events={}, prior_scales={}, protocols=[], history=Empty DataFrame
Columns: []
Index: [], first_timestamp=Timestamp('1970-01-01 00:00:00'), X=Empty DataFrame
Columns: []
Index: [], fit_kwargs={})

In [16]:
model.fit(df) 

07:13:35 - gloria - INFO - Starting optimization.


Gloria(model='poisson', sampling_period=Timedelta('1 days 00:00:00'), timestamp_name='ds', metric_name='y', capacity_name='', changepoints=Series([], Name: ds, dtype: datetime64[ns]), n_changepoints=0, changepoint_range=0.8, seasonality_prior_scale=3, event_prior_scale=3, changepoint_prior_scale=3, dispersion_prior_scale=3, interval_width=0.8, trend_samples=1000, _config={'fit': {'optimize_mode': 'MAP', 'use_laplace': False, 'capacity': None, 'capacity_mode': 'scale', 'capacity_value': 0.5}, 'predict': {'periods': 1, 'include_history': True}, 'load_data': {'source': '', 'dtype_kind': 'f'}}, model_backend=<gloria.models.Poisson object at 0x0000020933033B90>, vectorized=False, external_regressors={}, seasonalities={'monthly': Seasonality(name='monthly', prior_scale=3.0, period=3.472222222222222e-13, fourier_order=12)}, events={}, prior_scales={'Seasonality__delim__monthly__delim__odd__delim__1': 3.0, 'Seasonality__delim__monthly__delim__odd__delim__2': 3.0, 'Seasonality__delim__monthly__

In [18]:
df_crossval = cross_validation(model, horizon = '14 days', initial = '200 days' , period = '14 days') 
df_crossval

07:14:03 - gloria - INFO - Starting optimization.
07:14:03 - gloria - INFO - Starting optimization.
07:14:03 - gloria - INFO - Starting optimization.
07:14:04 - gloria - INFO - Starting optimization.
07:14:04 - gloria - INFO - Starting optimization.
07:14:04 - gloria - INFO - Starting optimization.
07:14:05 - gloria - INFO - Starting optimization.
07:14:05 - gloria - INFO - Starting optimization.
07:14:05 - gloria - INFO - Starting optimization.
07:14:06 - gloria - INFO - Starting optimization.
07:14:06 - gloria - INFO - Starting optimization.


,ds,yhat,observed_lower,observed_upper,y,cutoff
0,2020-07-30,17.351392,12.0,23.0,13,2020-07-29
1,2020-07-31,23.199565,17.0,29.0,24,2020-07-29
2,2020-08-01,15.324174,10.0,20.0,26,2020-07-29
3,2020-08-02,26.724328,20.0,33.0,39,2020-07-29
4,2020-08-03,16.425096,11.0,22.0,34,2020-07-29
...,...,...,...,...,...,...
149,2020-12-26,16.595572,11.0,22.0,40,2020-12-16
150,2020-12-27,22.220794,16.0,28.0,43,2020-12-16
151,2020-12-28,30.018356,23.0,37.0,24,2020-12-16
152,2020-12-29,19.590214,14.0,25.0,25,2020-12-16


In [31]:
from gloria import performance_metrics

In [32]:
performance_metrics(df_crossval, metric_name = 'y', timestamp_name = 'ds', horizon_period = '14 days')#, performance_metrics = ["rmse"]) 

,count,mse,rmse,mae,mape,smape,mdape,smdape,coverage
horizon,,,,,,,,,
14 days,154,551.624477,23.486687,11.107614,0.688609,0.402904,0.380241,0.334576,0.454545
